In [9]:
import os
import numpy as np
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split

# Dataset class

In [10]:
class FireDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(self.classes):
            class_folder = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_folder):
                self.image_paths.append(os.path.join(class_folder, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
batch_size = 64
train_size = 0.7
val_size = 0.15
test_size = 0.15

dataset_path = "fire_dataset/"  
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

fire_dataset = FireDataset(root_dir=dataset_path, transform=transform)
labels = np.array(fire_dataset.labels)


train_val_indices, test_indices = train_test_split(np.arange(len(fire_dataset)),test_size=test_size,stratify=labels)

val_relative_size = val_size / (train_size + val_size)
train_indices, val_indices = train_test_split(train_val_indices,test_size=val_relative_size,stratify=labels[train_val_indices])

train_subset = Subset(fire_dataset, train_indices)
val_subset = Subset(fire_dataset, val_indices)
test_subset = Subset(fire_dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# Iterate through the dataloader
for images, labels in train_loader:
    print(f"Batch size: {images.size(0)}")
    print(f"Image shape: {images.size()}")
    print(f"Labels: {labels}")
    break

Batch size: 64
Image shape: torch.Size([64, 3, 64, 64])
Labels: tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1])


# Printing the first image and its label for fun


In [4]:
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))  # CxHxW to HxWxC
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    img = std * img + mean  # Unnormalize
    img = np.clip(img, 0, 1)  # Clip to [0, 1] range
    return img


classes = fire_dataset.classes

data_iter = iter(train_loader)
images, labels = next(data_iter)

img = imshow(images[0])
label = labels[0].item()

# Display the image with its label
plt.figure(figsize=(4,4))
plt.imshow(img)
plt.title(f"Label: {classes[label]}")
plt.axis('off')  # Hide axis
plt.show()

NameError: name 'dataset' is not defined